# Quantum Chemistry Tutorial 2 - Excited states

1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
2. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
       
January 2024

## Introduction

L'une des applications marquantes de la chimie quantique, tant dans le milieu universitaire que dans l'industrie, est l'étude de l'interaction de la lumière avec la matière. L'absorption (resp. émission) d'un photon par une molécule peut promouvoir (resp. rétrograder) un électron d'un état électronique inférieur (resp. supérieur) à un état électronique d'énergie supérieure (resp. inférieure). La longueur d'onde des photons (c'est-à-dire l'énergie) requise pour que ces transitions se produisent est déterminée par la différence entre les deux états électroniques respectifs. Par conséquent, il est impératif de pouvoir calculer avec précision les énergies des états fondamental et excité afin d'étudier les interactions lumière/matière. Ces différences énergétiques jouent un rôle central dans de nombreuses technologies telles que les panneaux solaires, les diodes électroluminescentes (DEL ou en anglais, light-emitting diodes (LED)), les écrans et les colorants.

![Abs_emiss.png](Graphics/Abs_emiss.png)

Pour être plus concret, un colorant doit émettre de la lumière dans une région étroite du spectre visible pour être approprié à son objectif, c'est-à-dire qu'il doit présenter une longueur d'onde spécifique. Un autre exemple est celui des panneaux solaires, où le spectre d'absorption d'une molécule est ajusté via une fonctionnalisation chimique pour s'adapter au spectre d'émission solaire afin d'optimiser l'efficacité de la production d'énergie. Nous montrons ici un exemple de spectre pour la molécule BODIPY, une molécule largement utilisée pour les colorants fluorescents. BODIPY absorbe la lumière à une longueur d'onde inférieure (énergie plus élevée) et émet de la lumière à une longueur d'onde plus élevée (énergie plus faible). Pour calculer ce spectre, il faut calculer les énergies des états fondamental et excité et calculer leurs intensités. Le spectre d'absorption du BODIPY le plus simple est présenté ci-dessous. Différentes longueurs d'onde d'absorption et d'émission peuvent être ciblées en remplaçant les atomes d'hydrogène par différents groupes fonctionnels [J. Chimique. Phys. 155, 244102 (2021)](https://aip.scitation.org/doi/10.1063/5.0076787).

![BODIPY](Graphics/bodipy_absorption.png)

Comme il existe un très grand nombre de composés à considérer, la prévision des spectres d'absorption/émission UV-visible serait un atout précieux pour la communauté scientifique.

Pour parvenir à une compréhension complète de l'interaction de la lumière avec une molécule, la communauté de la chimie quantique a travaillé sur plusieurs algorithmes. En général, il faut calculer les structures électroniques moléculaires pertinentes pour prédire l'absorption/l'émission de la lumière UV. Ce notebook montre comment PySFC permet les calculs d'états excités en implémentant l'algorithme la TDDFT-TDA. Le cas d'utilisation ici est Li $_2$ par souci de simplicité. Ce calcul être étendu à des systèmes beaucoup plus grands tels que la molécule BODIPY ci-dessus.

Il convient de noter que même avec tous les états excités calculés, des effets non triviaux peuvent se produire (effet de solvatation, changement de géométrie, etc.) dans lesquels tous modifient la forme d'un spectre. Dans ce notebook, nous n'expliquons pas comment ces effets sont pris en compte, mais les calculs présentés ici constituent les premières étapes nécessaires vers le calcul des états excités.


## Théorie de la fonctionnelle de la densité - DFT

Dans la KS-DFT, proposé pour la première fois par Kohn et Sham, la densité électronique d'un système de référence sans interaction est utilisée pour représenter la densité du véritable système en interaction. En conséquence, la formulation informatique de KS-DFT ressemble à celle de la théorie de Hartree-Fock (HF), mais avec un potentiel de Fock effectif différent. Dans KS-DFT, l’énergie électronique totale est définie comme suit :

$$
    E = T_s + E_{\rm ext} + E_J + E_{\rm xc} .
$$
* $T_s$ est l'énergie cinétique sans interaction,
* $E_{\rm ext}$ est l'énergie due au potentiel externe, 
* $E_J$ est l'énergie coulombienne, 
* et $E_{\rm xc}$ est l'énergie d'échange-corrélation (*xc*). En pratique, $E_{\rm xc}$ est approché par une approximation fonctionnelle de la densité, qui elle-même peuvent être divisée en plusieurs classes le long de différents échelons de l'échelle de Jacob :
    * approximations de densité locale (par exemple LDA ; *xc* l'énergie dépend uniquement de la densité électronique, $\rho$),
    * approximations généralisées du gradient (GGA ; l'énergie *xc* dépend également du gradient de densité, $|\nabla\rho|$),
    * méta-GGA (l'énergie *xc* dépend aussi de la densité d'énergie cinétique et/ou de la densité laplacienne, $\sum_i |\nabla \psi_i|^2$, $\nabla^2\rho$ ; ce dernier n'est pas supporté dans PySCF pour le moment),
    * fonctionnelles de corrélation non locales (l'énergie *xc* implique une double intégrale)
    * fonctionnelles de densité hybride (une fraction de l'échange exact est utilisée), et
    * fonctionnelles de densité corrigées à longue portée (l'échange exact est utilisé avec un noyau d'interaction modifié)

La minimisation variationnelle de l'énergie totale par rapport à la densité donne les équations KS pour les orbitales de référence sans interaction, comparables à la théorie HF, et celles-ci ont la même forme générale que les équations de Fock dans la SCF. Cependant, l'échange exact, $\hat{K}$, est remplacé par le potentiel *xc*, $\hat{v}_{\rm xc}=\frac{\delta E_{\rm xc}}{\delta \rho}$. Pour les calculs hybrides et méta-GGA, PySCF utilise le formalisme KS généralisé, dans lequel les équations KS généralisées minimisent l'énergie totale par rapport aux orbitales elles-mêmes.

![DFT_Flowchart.jpg](Graphics/DFT_Flowchart.jpg)

## Théorie de la fonctionnelle de la densité dépendante du temps - TDDFT

En utilisant la théorie des perturbations dépendant du temps du premier ordre au sein de la théorie KS-DFT, on obtient les équations non Hermitiennes TDDFT pour les énergies d'excitation,
$$
     \begin{pmatrix}
         \mathbf{A} & \mathbf{B} \\
         \mathbf{B}^\ast & \mathbf{A}^\ast
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X} \\ \mathbf{Y}
     \end{pmatrix} = \omega
     \begin{pmatrix}
         \mathbf{1} & \mathbf{0} \\
         \mathbf{0} & -\mathbf{1}
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X} \\ \mathbf{Y}
     \end{pmatrix},
$$    
où 
* $\mathbf{A}$ et $\mathbf{B}$ sont les orbitales les Hessiennes qui apparaissent également dans l'analyse de stabilité pour les états de référence ,
* $\omega$ est l'énergie d'excitation, et
* $\mathbf{X}$ et $\mathbf{Y}$ représentent la réponse de la matrice de densité ou l'opérateur statistique.

Dans les cas où le système possède un état fondamental dégénéré ou présente des instabilités triplet, les algorithmes utilisés pour résoudre le
les équations ci-dessus peuvent être instables. Ceci peut être résolu en appliquant l'approximation de Tamm-Dancoff (TDA), où l'on néglige simplement les matrices $\mathbf{B}$ et $\mathbf{Y}$. Il en résulte un problème Hermitien aux valeurs propres
$$
\mathbf{AX} = \omega\mathbf{X}.
$$

Il est à noter que lorsque TDA est appliquée, dans la TDDFT, la réponse linéaire du potentiel de corrélation d'échange (XC) conduit à une dérivée d'ordre 2 de la fonctionnelle XC, qui n'apparaît pas dans la DFT de l'état fondamental.


## Propriétés des transitions

Les énergies d’excitation ne rendent compte ne sont pas suffisantes comprendre la physique du problème. Nous avons besoin de moments de transition pour comprendre si une excitation est autorisée (lumineuse) ou non (sombre).

Les énergies d'excitation et les vecteurs propres peuvent être utilisés pour calculer les moments de transition, tels que les moments dipolaires de transition électrique et magnétique, et les intensités spectroscopiques, telles que les forces d'oscillateur et les forces de rotation.
Par exemple, le moment dipolaire électrique de transition dans la jauge de longueur est :

$$
  f = \frac23 \omega_n \sum_{u=x,y,z}\sum_{ia}|(\mathbf{X}_n+\mathbf{Y}_n)_{ia}\mu_{ai, u}|^2.
$$
